In [1]:
from lib.config import Config_f
from lib.data_set import Features
from lib.model import SimpleModel
from lib import utils

In [2]:
import ctypes
import pandas as pd
import numpy as np
import pywt
import time
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest,f_classif,chi2,mutual_info_classif,VarianceThreshold,RFE,SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score

from tensorflow.keras import layers as KL
from tensorflow.keras import models as KM
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow import keras
import tensorflow as tf

## File name read

In [3]:
# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)
player = ctypes.windll.kernel32

ind = df2.iloc[1].isna()
files = np.concatenate([np.array(df.columns),np.array('normal/'+df2.columns[ind])])

## Configuration

In [17]:
# Override the base class of Config and Features for ANN Model
class ANN_Config(Config_f):
    NAME = 'ANN'
    NUM_CLASSES = 2
    EPOCHS = 300
    BATCH_SIZE = 32
    CLASS_WEIGHTS = None
    COST_SENSITIVE = False
    
    FN_LP = 300
    DETREND_LAMBDA = 50
    TEST_FILES = files[[5,30,31,32,33,34,35]]
    
    
class ANN_dataset(Features):
    
    def __init__(self,config):
        super(ANN_dataset,self).__init__(config)
        self.config = config
        

In [18]:
# Generate ANN configuration
config = ANN_Config()
config.display()


Configurations:
BATCH_SIZE                     32
BINS                           3
CHANNELS                       ['LEFT_TA', 'LEFT_TS', 'LEFT_BF', 'LEFT_RF', 'RIGHT_TA', 'RIGHT_TS', 'RIGHT_BF', 'RIGHT_RF']
CLASS_WEIGHTS                  None
COST_SENSITIVE                 False
DETREND_LAMBDA                 50
DROP_WITH_ZSCORE               None
EPOCHS                         300
FEATURES_LIST                  ['IEMG', 'SSI', 'WL', 'ZC', 'ku', 'SSC', 'skew', 'Acti', 'AR', 'HIST', 'MDF', 'MNF', 'mDWT']
FN_HP                          None
FN_IR                          False
FN_LP                          300
LEVEL_DWT                      3
NAME                           ANN
NUM_CLASSES                    2
NUM_MF                         3
N_ENV                          20
RANGES                         (-3, 3)
RECT                           False
REMOVE_FREQS                   True
SAME_LABEL                     True
SAVE                           False
SCALE                        

## Data generate

In [19]:
data = ANN_dataset(config)

In [20]:
# Choose features to use
data.feature_list = ['IEMG', 'SSI', 'WL', 'ZC', 'ku', 'SSC', 'skew', 'Acti', 'AR', 'HIST', 'MDF', 'MNF', 'mDWT']

# Load data from files
data.load_data(files)

# Extract features from data
data.extract_features()

X_train,Y_train,_ = data.train_set
X_valid,Y_valid,_ = data.valid_set
X_test, Y_test, _ = data.test_set

skip
skip
3/174: G06_FoG_trial_1_emg.csv
4/174: G06_FoG_trial_2_emg.csv
5/174: G06_FoG_trial_3_emg.csv
6/174: G07_Freezing_Trial1_trial_1_emg.csv
7/174: G08_FoG_1_trial_1_emg.csv
8/174: G08_FoG_2_trial_1_emg.csv
9/174: G11_FoG_trial_1_emg.csv
10/174: G11_FoG_trial_2_emg.csv
11/174: P379_M050_2_OFF_A_FoG_trial_1_emg.csv
12/174: P379_M050_2_OFF_A_FoG_trial_2_emg.csv
13/174: P379_M050_2_OFF_A_FoG_trial_3_emg.csv
14/174: P379_M050_2_OFF_B_FoG_trial_1_emg.csv
15/174: P379_M050_2_OFF_B_FoG_trial_2_emg.csv
16/174: P379_M050_2_OFF_B_FoG_trial_3_emg.csv
17/174: P551_M050_2_A_FoG_trial_1_emg.csv
18/174: P551_M050_2_B_FoG_trial_1_emg.csv
19/174: P551_M050_2_B_FoG_trial_2_emg.csv
20/174: P812_M050_2_B_FoG_trial_1_emg.csv
21/174: P812_M050_2_B_FoG_trial_2_emg.csv
22/174: normal/G02_Walking_trial_1_emg.csv
23/174: normal/G03_Walking_trial_1_emg.csv
24/174: normal/G03_Walking_trial_2_emg.csv
25/174: normal/G05_Walking_struct_fixed_trial_1_emg.csv
26/174: normal/G05_Walking_struct_fixed_trial_2_emg.cs

171/174: normal/P940_MSham_A_Walking_trial_6_emg.csv
172/174: normal/P940_MSham_B_Walking_trial_2_emg.csv
173/174: normal/P940_MSham_B_Walking_trial_4_emg.csv
174/174: normal/P940_MSham_B_Walking_trial_6_emg.csv
threshold_WAMP:1.0, threshold_ZC:0.0, threshold_SSC:0.0, bins:3, ranges:(-3,3), num_mf:3, wavelet: db7, level: 3
['IEMG', 'SSI', 'WL', 'ZC', 'ku', 'SSC', 'skew', 'Acti', 'AR', 'HIST', 'MDF', 'MNF', 'mDWT']
threshold_WAMP:1.0, threshold_ZC:0.0, threshold_SSC:0.0, bins:3, ranges:(-3,3), num_mf:3, wavelet: db7, level: 3
['IEMG', 'SSI', 'WL', 'ZC', 'ku', 'SSC', 'skew', 'Acti', 'AR', 'HIST', 'MDF', 'MNF', 'mDWT']
threshold_WAMP:1.0, threshold_ZC:0.0, threshold_SSC:0.0, bins:3, ranges:(-3,3), num_mf:3, wavelet: db7, level: 3
['IEMG', 'SSI', 'WL', 'ZC', 'ku', 'SSC', 'skew', 'Acti', 'AR', 'HIST', 'MDF', 'MNF', 'mDWT']


## Model

In [21]:
# Override base class of SimpleMode for ANN
class ANN_Model(SimpleModel):
    
    def build(self,config):
        
        reg = keras.regularizers.l1(0)
        acti = 'elu'
        drop = 0.2
        init = 'glorot_normal'

        model = KM.Sequential()
        model.add(KL.Dense(128,
                               kernel_initializer=init,
                               kernel_regularizer = reg,
                               #use_bias=False
                         ))
        model.add(KL.Activation(acti))
        model.add(KL.Dropout(drop))

        model.add(KL.Dense(64,
                               kernel_initializer=init,
                               kernel_regularizer = reg,
                              # use_bias=False
                         ))
        model.add(KL.Activation(acti))
        model.add(KL.Dropout(drop))

        model.add(KL.Dense(32,
                               kernel_initializer=init,
                               kernel_regularizer = reg,
                               #use_bias=False
                         ))
        model.add(KL.Activation(acti))
        model.add(KL.Dropout(drop))

        model.add(KL.Dense(16,
                               kernel_initializer=init,
                               kernel_regularizer = reg,
                               #use_bias=False
                         ))
        model.add(KL.Activation(acti))
        model.add(KL.Dropout(drop))

        model.add(KL.Dense(config.NUM_CLASSES,activation='softmax'))

        
        if config.COST_SENSITIVE:
            self.cost_matrix = config.COST_MATRIX
            model.compile(loss=self.sparse_cost_sensitive_loss, optimizer="adam", metrics=['accuracy'])
            print('Using cost sensitive with cost matrix:\n',np.array(self.cost_matrix))
        else:
            model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
            if config.CLASS_WEIGHTS != None:
                print('Using categorical crossentropy with class weights:\n',config.CLASS_WEIGHTS)
            else:
                print('Using categorical crossentropy without class weights.')
        
        return model
    
    def train(self, train_dataset, val_dataset, oh, transformer=None, callbacks=None):
        
        self.X_train = train_dataset[0]
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(train_dataset[0])
        
        scaler.fit(np.concatenate([train_dataset[0],val_dataset[0]]))
        X_val = scaler.transform(val_dataset[0])
        
        if transformer != None:
            self.transformer = transformer
            self.transformer.fit(X_train,np.ravel(oh.inverse_transform(train_dataset[1])))
            X_train = self.transformer.transform(X_train)
            X_val = self.transformer.transform(X_val)
        else:
            self.transformer = None

        self.simple_model.fit(X_train,
                              train_dataset[1],
                              validation_data=(X_val,val_dataset[1]),
                              epochs=self.config.EPOCHS,
                              batch_size=self.config.BATCH_SIZE,
                              class_weight=self.config.CLASS_WEIGHTS,
                              callbacks=callbacks,
                              shuffle=True)

    def sparse_cost_sensitive_loss (self,y_true,y_pred):
        cost_matrix = self.cost_matrix
        batch_cost_matrix = tf.nn.embedding_lookup(cost_matrix, tf.argmax(y_true,axis=1))
        eps = 1e-6
        probability = tf.clip_by_value(y_pred, eps, 1-eps)
        cost_values = tf.math.log(1-probability)*batch_cost_matrix
        loss = tf.reduce_mean(-tf.reduce_sum(cost_values, axis=1))
        return loss
    
    def model_metrics(self,data,label):
        pred = self.predict(data)
        acc = accuracy_score(np.argmax(label,axis=1),np.argmax(pred,axis=1))
        cm = confusion_matrix(np.argmax(label,axis=1),np.argmax(pred,axis=1))
        f1 = f1_score(np.argmax(label,axis=1),np.argmax(pred,axis=1),average='macro')
        return acc,cm,f1

## Data split

In [22]:
# data split and processing for model
class_id = [1,2,6]
binary = True
x_train,y_train,x_valid,y_valid,x_test,y_test,oh = utils.data_split_oh((X_train,X_valid,X_test),
                                                                    (Y_train,Y_valid,Y_test),
                                                                    class_id,
                                                                    binary,
                                                                    random_state = 555)

## Model training

In [26]:
config.EPOCHS = 300
config.NUM_CLASSES = 2
config.CLASS_WEIGHTS = None#{0:1,1:5}

config.COST_MATRIX = tf.constant([[0,1.],
              [5,0]])

pca = PCA(n_components=80,copy=True)
sfm = SelectFromModel(GradientBoostingClassifier(),max_features=80)
rfe = RFE(estimator=LogisticRegression(max_iter=10000), n_features_to_select=80)
vt = VarianceThreshold(threshold=0.01)

if binary:
    config.COST_SENSITIVE = True
    config.NUM_CLASSES = 2
    transformer=rfe
else:
    config.COST_SENSITIVE = False
    config.NUM_CLASSES = len(class_id)
    transformer=pca

# Generate ANN Model
ANN_model = ANN_Model('ANN',config,'./model/ANN/')

Using cost sensitive with cost matrix:
 [[ 0.  1.]
 [10.  0.]]


In [27]:
early_stopping = keras.callbacks.EarlyStopping(patience = 20,
                                             monitor = 'val_loss', 
                                             #baseline = 0.9,
                                             restore_best_weights=True)

ANN_model.train((x_train,y_train),
                (x_valid,y_valid),
                oh,
                transformer,
                [early_stopping])

Epoch 1/300
173/173 [==============================] - 2s 5ms/step - loss: 1.1599 - accuracy: 0.6642 - val_loss: 0.6068 - val_accuracy: 0.7518
Epoch 2/300
173/173 [==============================] - 0s 3ms/step - loss: 0.5240 - accuracy: 0.8730 - val_loss: 0.3870 - val_accuracy: 0.8737
Epoch 3/300
173/173 [==============================] - 1s 3ms/step - loss: 0.4406 - accuracy: 0.8936 - val_loss: 0.4519 - val_accuracy: 0.8133
Epoch 4/300
173/173 [==============================] - 1s 4ms/step - loss: 0.4073 - accuracy: 0.8958 - val_loss: 0.3661 - val_accuracy: 0.8606
Epoch 5/300
173/173 [==============================] - 1s 5ms/step - loss: 0.3728 - accuracy: 0.9107 - val_loss: 0.2736 - val_accuracy: 0.9477
Epoch 6/300
173/173 [==============================] - 1s 4ms/step - loss: 0.3769 - accuracy: 0.9105 - val_loss: 0.2709 - val_accuracy: 0.9554
Epoch 7/300
173/173 [==============================] - 0s 3ms/step - loss: 0.3725 - accuracy: 0.9166 - val_loss: 0.3225 - val_accuracy: 0.8759

Epoch 58/300
173/173 [==============================] - 1s 3ms/step - loss: 0.1840 - accuracy: 0.9541 - val_loss: 0.2246 - val_accuracy: 0.9744
Epoch 59/300
173/173 [==============================] - 1s 4ms/step - loss: 0.2452 - accuracy: 0.9417 - val_loss: 0.4910 - val_accuracy: 0.8323
Epoch 60/300
173/173 [==============================] - 0s 3ms/step - loss: 0.2051 - accuracy: 0.9424 - val_loss: 0.1721 - val_accuracy: 0.9499
Epoch 61/300
173/173 [==============================] - 0s 2ms/step - loss: 0.1873 - accuracy: 0.9485 - val_loss: 0.5071 - val_accuracy: 0.8084
Epoch 62/300
173/173 [==============================] - 0s 2ms/step - loss: 0.2029 - accuracy: 0.9521 - val_loss: 0.2105 - val_accuracy: 0.9369
Epoch 63/300
173/173 [==============================] - 1s 3ms/step - loss: 0.2130 - accuracy: 0.9493 - val_loss: 0.1597 - val_accuracy: 0.9608
Epoch 64/300
173/173 [==============================] - 0s 2ms/step - loss: 0.1669 - accuracy: 0.9603 - val_loss: 0.1647 - val_accuracy:

## Model evaluation

In [28]:
acc_train,cm_train,f1_train = ANN_model.model_metrics(x_train,y_train)
acc_valid,cm_valid,f1_valid = ANN_model.model_metrics(x_valid,y_valid)
acc_test,cm_test,f1_test = ANN_model.model_metrics(x_test,y_test)
print('acc_train: %f\nf1_train: %f\nconfusion_matrix:\n'%(acc_train,f1_train),cm_train,'\n')
print('acc_valid: %f\nf1_valid: %f\nconfusion_matrix:\n'%(acc_valid,f1_valid),cm_valid,'\n')
print('acc_test: %f\nf1_test: %f\nconfusion_matrix:\n'%(acc_test,f1_test),cm_test)

acc_train: 0.964240
f1_train: 0.903098
confusion_matrix:
 [[4844  197]
 [   0  468]] 

acc_valid: 0.953729
f1_valid: 0.887379
confusion_matrix:
 [[1581   83]
 [   2  171]] 

acc_test: 0.949735
f1_test: 0.935619
confusion_matrix:
 [[268  17]
 [  2  91]]


In [34]:
files[[7,40,41,42,43,44,45]]

array(['G08_FoG_2_trial_1_emg.csv',
       'normal/P231_M050_B_Walking_trial_6_emg.csv',
       'normal/P231_M100_2_A_FoG_trial_3_emg.csv',
       'normal/P231_M100_2_A_Walking_trial_4_emg.csv',
       'normal/P231_M100_2_A_Walking_trial_6_emg.csv',
       'normal/P231_M100_ON_A_Walking_trial_2_emg.csv',
       'normal/P231_M100_ON_A_Walking_trial_4_emg.csv'], dtype=object)